In [48]:
import tabula
import pandas as pd
import os
import re

In [6]:
pdfs = [x for x in os.listdir() if x.endswith('.pdf')]

In [5]:
os.chdir(os.path.join(os.getcwd(),'csv_not_in/'))

In [39]:
table_list = []

for pdf in pdfs:
    tables = tabula.read_pdf(pdf, multiple_tables=True, pages='all')
    for table in tables:
        table_list.append([table, pdf])

Got stderr: mar 16, 2020 10:03:16 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored
mar 16, 2020 10:03:16 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri-Bold are not implemented in PDFBox and will be ignored
mar 16, 2020 10:03:16 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored
mar 16, 2020 10:03:16 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
mar 16, 2020 10:03:17 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri-BoldItalic are not implemented in PDFBox and will be ignored

Got stderr: mar 16, 2020 10:03:18 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri-Bold 

In [56]:
complex_tables = []
simple_tables = []
pattern = 'Podmiot'
for l in table_list:
    text = list(l[0].columns)[0]
    if re.search(pattern,text):
        simple_tables.append(l[0])
    else:
        complex_tables.append(l[0])


In [58]:
simple_tables[0]

2,Podmiot wykonujący działalność leczniczą 1,NaN
3,nazwa:,SZPITAL KLINICZNY IM. KAROLA JONSCHERA UNIWERS...
4,adres: miejscowość/ kod pocztowy/ ulica i nr domu,"Poznań 60-572, ul. Szpitalna 27/33"
5,telefon/ telefony:,61 849 12 00
6,identyfikator REGON,000288863
7,Miejsce udzielania świadczeń 1 *,NaN
8,nazwa:,"Oddział VIII Chorób Zakaźnych i Neurologii, Pe..."
9,adres: miejscowość/ kod pocztowy/ ulica i nr domu,"Poznań 60-572, ul. Szpitalna 27/33"
10,telefon/ telefony,504 907 994


In [4]:
exceptions = []
def transform(df):
    
    if len(df.columns) == 2:
        df.columns = [1,2]
        df.set_index(1)
        col = ['nazwa', 'adres', 'telefon']
        try:
            df.insert(0,'index',col)
            df.drop(columns=1, inplace=True)
            df.set_index('index', inplace=True)
            df = df.transpose()
            return df
        except ValueError:
            exceptions.append(df)

In [6]:
hospital_list = []
for tupl in table_list:
    frame = tupl[0].loc[:2]
    hospital = transform(frame)
    hospital_list.append(hospital)
    

/home/ms/.local/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [10]:
df = pd.concat(hospital_list)

In [22]:
df = df.reset_index()

In [25]:
df.drop('index', axis=1, inplace=True)

In [36]:
df.loc[0]

index
nazwa      SZPITAL OGÓLNY IM. DR WITOLDA GINELA\rW GRAJEWIE
adres                 GRAJEWO 19-200, KONSTYTUCJI 3 MAJA 34
telefon                                       86 272 232 71
Name: 0, dtype: object

In [65]:
col = ['nazwa', 'adres', 'telefon']
ex1 = exceptions[2].transpose()
ex1.insert(0, 'nazwa', ['nazwa', 'Uniwersytecki Dziecięcy Szpital Kliniczny im. L. Zamenhofaw Białymstoku'])
ex1.columns = col
ex1 = ex1.loc[2:2].reset_index()
ex1.drop('index', axis=1, inplace=True)
ex1

,nazwa,adres,telefon
0,Uniwersytecki Dziecięcy Szpital Kliniczny im. ...,"BIAŁYSTOK 15-274, JERZEGO\rWASZYNGTONA, 17",85 745 06 93


In [69]:
df = df.append(ex1, ignore_index=True)

In [70]:

col = ['nazwa', 'adres', 'telefon']
ex1 = exceptions[4].transpose()
ex1.insert(0, 'nazwa', ['nazwa', 'Podhalański Szpital Specjalistyczny'])
ex1.columns = col
ex1 = ex1.loc[2:2].reset_index()
ex1.drop('index', axis=1, inplace=True)
ex1

,nazwa,adres,telefon
0,Podhalański Szpital Specjalistyczny,"ul. Szpitalna 14, 34-400 Nowy Targ",NaN


In [71]:
df = df.append(ex1, ignore_index=True)

In [77]:
df.loc[132].telefon = '18 263 30 01'

In [78]:
df

index,nazwa,adres,telefon
0,SZPITAL OGÓLNY IM. DR WITOLDA GINELA\rW GRAJEWIE,"GRAJEWO 19-200, KONSTYTUCJI 3 MAJA 34",86 272 232 71
1,SAMODZIELNY PUBLICZNY ZAKŁAD OPIEKI\rZDROWOTNE...,"AUGUSTÓW, 16-300, SZPITALNA 12",87 644 42 84
2,SAMODZIELNY PUBLICZNY ZAKŁAD OPIEKI\rZDROWOTNE...,"HAJNÓWKA, 17-200 DOC. ADAMA\rDOWGIRDA, 9",85 682 92 02
3,SAMODZIELNY PUBLICZNY ZAKŁAD OPIEKI\rZDROWOTNE...,"BIELSK PODLASKI, 17-100, KLESZCZELOWSKA\r1",85 833 43 30
4,SZPITAL WOJEWÓDZKI IM. DR. LUDWIKA\rRYDYGIERA ...,"SUWAŁKI, 16-400, SZPITALNA, 60",87 562 94 21
...,...,...,...
128,WOJEWÓDZKA STACJA POGOTOWIA RATUNKOWEGO W\rRZE...,"35-026 Rzeszów, ul. Księcia Józefa Poniatowski...",17 85 26 253
129,BIESZCZADZKIE POGOTOWIE RATUNKOWE SPZOZ W\rSANOKU,"38-500 Sanok, ul. Jezierskiego 21",13 46 28 460
130,KLINICZNY SZPITAL WOJEWÓDZKI NR 2 IM. ŚW.JADWI...,"35-301 Rzeszów, ul. Lwowska 60",17 86 64 000
131,Uniwersytecki Dziecięcy Szpital Kliniczny im. ...,"BIAŁYSTOK 15-274, JERZEGO\rWASZYNGTONA, 17",85 745 06 93


In [81]:
df.nazwa = df.nazwa.str.upper()

In [83]:
df.adres = df.adres.str.upper()

In [86]:
df.nazwa = df.nazwa.str.replace('\r', ' ')

In [87]:
df.adres = df.adres.str.replace('\r', ' ')

In [93]:
df.isnull().values.any()

True

In [102]:
df.loc[df.adres.isnull()==True]

index,nazwa,adres,telefon
39,42 655 76 71,NaN,Zespół Transportu Sanitarnego


In [98]:
df.loc[122].adres = 'Żeromskiego 22, 39-300 Mielec'

In [99]:
df.loc[122].telefon = '17 78 00 100'

In [101]:
df.loc[49].adres = 'Sosnowiec 41-200 ul. Teatralna 9'
df.loc[49].telefon = '32 363 60 32'

In [104]:
df.to_csv('df.txt')